In [ ]:
# app.py

!pip install streamlit
import streamlit as st
import pandas as pd
import numpy as np
import pickle

# ---------------- Cargar modelos y utilidades ----------------

st.set_page_config(page_title="Predicción de Desistido en Leads Inmobiliarios", layout="centered")

st.title("🏗️ Predicción de Desistido en Leads Inmobiliarios")

modelo_path = 'modelos_finales_corregido.pkl'

with open(modelo_path, 'rb') as archivo:
    modelos_cargados = pickle.load(archivo)

modelTree = modelos_cargados['DecisionTree']
modelKnn = modelos_cargados['KNN']
modelNN = modelos_cargados['NeuralNetwork']
scaler = modelos_cargados['Scaler']
variables = modelos_cargados['Variables']

# ---------------- Entradas de usuario ----------------

st.header("🔹 Ingrese la información del lead:")

# Variables numéricas
EdadComprador = st.number_input('Edad del Comprador', min_value=18, max_value=99, value=35)
porc_part = st.number_input("Porcentaje Participación Comprador", min_value=1, max_value=100, value=50)
valor_cuota_sep = st.number_input("Valor Cuota Separación", min_value=500000, max_value=10000000, value=1500000)
valor_negocio = st.number_input("Valor del Negocio", min_value=30000000, max_value=1000000000, value=120000000)
valor_credito = st.number_input("Valor del Crédito", min_value=0, max_value=1000000000, value=80000000)
valor_cuota_ini = st.number_input("Valor Cuota Inicial", min_value=0, max_value=1000000000, value=40000000)

# Variables categóricas (desplegables)
# Listados según tus columnas entrenadas
proyecto = st.selectbox("Proyecto", [
    "Proyecto_'BULEVAR DEL ALCAZAR'", "Proyecto_'BULEVAR LIVING'",
    "Proyecto_'CAMINO DEL VIENTO'", "Proyecto_'DISTRITO PLAZA'",
    "Proyecto_'NATIVO AGUA'", "Proyecto_'NATIVO ARENA'",
    "Proyecto_'NATIVO FLORA'", "Proyecto_'NATIVO MADERA'",
    "Proyecto_'RESERVA DEL BOSQUE'", "Proyecto_'RESERVA DEL PARQUE'",
    "Proyecto_'RESERVA SAN JUAN'", "Proyecto_BANTUE",
    "Proyecto_ECOH", "Proyecto_ENSENADA", "Proyecto_ENTREBOSQUES",
    "Proyecto_ENTREHOJAS", "Proyecto_FONT", "Proyecto_FONTANAR",
    "Proyecto_HERBARIO", "Proyecto_IROKO", "Proyecto_KUNA",
    "Proyecto_NUVICA", "Proyecto_PACIFICA", "Proyecto_PBB-CAMBULO",
    "Proyecto_PBB-EUCALIPTO", "Proyecto_PBB-YARUMO",
    "Proyecto_SOLANA", "Proyecto_TERRAH"
])

tiponegocio = st.selectbox("Tipo de Negocio", [
    'TipoNegocio_NEGOCIOPRINCIPAL', 'TipoNegocio_NEGOCIOSECUNDARIO',
    'TipoNegocio_OTROSCONCEPTOS', 'TipoNegocio_REFORMAS'
])

estado_civil = st.selectbox("Estado Civil", [
    'EstadoCivilComprador_Divorciadoconsociedadconyugald.Yl.',
    'EstadoCivilComprador_Soltero',
    'EstadoCivilComprador_Unionlibre',
    'EstadoCivilComprador_Viudo'
])

profesion = st.selectbox("Profesión Agrupada", [
    "Profesion_Agrupada_'Administrador De Negocios'",
    "Profesion_Agrupada_'Arquitectura Y Diseño'",
    "Profesion_Agrupada_'Ciencias Agropecuarias'",
    "Profesion_Agrupada_'Contaduría Y Finanzas'",
    "Profesion_Agrupada_'Empresario / Independiente'",
    'Profesion_Agrupada_Comunicación',
    'Profesion_Agrupada_Derecho',
    'Profesion_Agrupada_Educación',
    'Profesion_Agrupada_Empleado',
    'Profesion_Agrupada_Estudiante',
    'Profesion_Agrupada_Hogar',
    'Profesion_Agrupada_Ingeniería',
    'Profesion_Agrupada_Otra',
    'Profesion_Agrupada_Pensionado',
    'Profesion_Agrupada_Salud'
])

descripcion_ampliacion = st.selectbox("Descripción Ampliación", [
    'DescripciónAmpliación_Campestre',
    'DescripciónAmpliación_Full',
    'DescripciónAmpliación_NoTiene'
])

# ---------------- Preparación del dataframe ----------------

# Inicializar todas las columnas en 0
input_dict = {col: 0 for col in variables}

# Asignar valores numéricos
input_dict['EdadComprador'] = EdadComprador
input_dict["'Porcentaje Participación Comprador'"] = porc_part
input_dict['ValorCuotaSeparacion'] = valor_cuota_sep
input_dict['Valordelnegocio'] = valor_negocio
input_dict['NegocioValorCrédito'] = valor_credito
input_dict['NegocioValorCuotaInicial'] = valor_cuota_ini

# Activar la categoría seleccionada con valor 1
for col in [proyecto, tiponegocio, estado_civil, profesion, descripcion_ampliacion]:
    if col in input_dict:
        input_dict[col] = 1

# Crear DataFrame
df_input = pd.DataFrame([input_dict])

# Normalizar numéricas
numeric_features = [
    'EdadComprador',
    "'Porcentaje Participación Comprador'",
    'ValorCuotaSeparacion',
    'Valordelnegocio',
    'NegocioValorCrédito',
    'NegocioValorCuotaInicial'
]
df_input[numeric_features] = scaler.transform(df_input[numeric_features])

# ---------------- Predicción ----------------

if st.button("Predecir"):
    pred_tree = modelTree.predict(df_input)[0]
    pred_knn = modelKnn.predict(df_input)[0]
    pred_nn = modelNN.predict(df_input)[0]

    st.subheader("🔹 Resultados de Predicción:")
    st.write(f"🌳 **Árbol de Decisión:** {'Desiste' if pred_tree == 1 else 'No desiste'}")
    st.write(f"🔹 **KNN:** {'Desiste' if pred_knn == 1 else 'No desiste'}")
    st.write(f"🧠 **Red Neuronal:** {'Desiste' if pred_nn == 1 else 'No desiste'}")

    st.success("✅ Predicción realizada correctamente. Usa estos resultados para la gestión de leads.")

